## Defining the linear progam

Decision Variables:
- $x_{i}$ represents allocation of portfolio to asset $i$.

Contraints:
- $\sum_{i} x_i = 1$
- $x_i \ge 0 \forall i$

Objective function:
$$ \max_x \sum_{j} p_j \times (x^T \mu_{j} - \lambda \times x^T \Sigma_j x)$$

Constants being used:
- $\mu_j$, $\Sigma_j$ are expected returns and covariance matrix under scenario $j$
- $p_j$ is the probability of being in scenario $j$
- $\lambda$ is a constant representing risk tolerance.

In [96]:
using JuMP, Gurobi, LinearAlgebra, CSV

In [97]:
m = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [98]:
num_assets = 4

@variable(m, x[1:num_assets])

@constraint(m, non_neg[i= 1:num_assets], x[i] >= 0)
@constraint(m, sum(x[i] for i=1:num_assets) == 1)

x[1] + x[2] + x[3] + x[4] == 1.0

In [99]:
CSV.read("data/Weak_ExpReturns.csv", header=0)

,Column1,Column2
,String,Float64
1,SPY_Return,-0.252131
2,US_Corp_Return,0.0286921
3,TreasInd_Return,0.00454707
4,Cash_Return,0.00104796


In [100]:
#Have to load the data.
weak_mu = CSV.read("data/Weak_ExpReturns.csv", header=0)[:, 2]
weak_sigma = convert(Matrix, CSV.read("data/Weak_CovMat.csv", header=1)[:, 2:num_assets+1])

normal_mu = CSV.read("data/Normal_ExpReturns.csv", header=0)[:, 2]
normal_sigma = convert(Matrix, CSV.read("data/Normal_CovMat.csv", header=1)[:, 2:num_assets+1])

strong_mu = CSV.read("data/Strong_ExpReturns.csv", header = 0)[:, 2]
strong_sigma = convert(Matrix, CSV.read("data/Strong_CovMat.csv", header=1)[:, 2:num_assets+1])

4×4 Array{Float64,2}:
  0.0104423     0.00151692   -0.000407407   5.55994e-6
  0.00151692    0.000951612  -0.000106923   2.99232e-7
 -0.000407407  -0.000106923   0.000440469  -3.49966e-6
  5.55994e-6    2.99232e-7   -3.49966e-6    1.25276e-7

In [101]:
probs = CSV.read("data/theta0_Probs.csv")[1, 2:num_assets]
normal_prob = probs[1]

strong_prob = probs[2]

weak_prob = probs[3]

0.14148293469399878

In [102]:
transpose(x)*weak_mu

-0.2521308999583641 x[1] + 0.02869206593346961 x[2] + 0.00454706660545963 x[3] + 0.0010479638958430766 x[4]

In [103]:
lambda = 3
@objective(m, Max, weak_prob*(transpose(x)*weak_mu - lambda * transpose(x)*weak_sigma*x) + normal_prob*(transpose(x)*normal_mu - lambda * transpose(x)*normal_sigma*x) + strong_prob*(transpose(x)*strong_mu - lambda*transpose(x)*strong_sigma*strong_mu))

-0.031561399971499496 x[1]² - 0.010930518463054999 x[1]*x[2] + 0.0008806604477534773 x[1]*x[3] + 0.00013526906297120465 x[1]*x[4] - 0.009157535479433031 x[2]² - 0.001179017185037863 x[2]*x[3] + 5.42808680954073e-5 x[2]*x[4] - 0.0016413665455904272 x[3]² + 7.061484922643777e-7 x[3]*x[4] - 2.2291096357123482e-6 x[4]² + 0.06989397512662873 x[1] + 0.06270921260255818 x[2] + 0.004232205276027821 x[3] + 0.0007645809344731123 x[4]

In [104]:
optimize!(m)

Academic license - for non-commercial use only
Optimize a model with 5 rows, 4 columns and 8 nonzeros
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 7e-02]
  QObjective range [1e-06, 6e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 4 columns, 4 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.24933924e+05  1.25344544e+05  3.50e+03 1.40e-01  1.00e+06     0s
   1  -3.34482067e+03  3.64030428e+03  1.17e+02 2.68e-08  3.49e+04     0s
   2   4.06798817e-02  2.97041711e+02  7.66e-03 1.76e-12  7.

In [105]:
#SPY, US_Corp, TreasIndex, Cash
value.(x)

4-element Array{Float64,1}:
 0.24454664832192444   
 0.7554533516503955    
 1.5646605855797518e-11
 1.2033384613846213e-11